In [1]:
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import glob
from tqdm import tqdm
import gensim
import multiprocessing
import random
nltk.download('stopwords')
import pandas as pd
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
#Clean the data function
stopwords_list = stopwords.words('english')
cores=multiprocessing.cpu_count()
import string

def clean_data(w):
    w = w.lower()                                                                             #Lower casing
    w=re.sub(r'[^\w\s]','',w)                                                                 #Remove other special characters
    w=re.sub(r"([0-9])", r" ",w)                                                              #Remove numbers       
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]  #Remove stop and short words
    return " ".join(clean_words)  


    

In [3]:
# Please, always put this cell on top of any Notebook you make.
# It replaces the former cells with Google Drive mounting and change directory.
# Using the code in this cell ensures that the Notebook is easy to run on Google Colab and on Jupyter Lab alike.
import os

def init_google_colab(gpath = '/content/gdrive/MyDrive/Assignment-Semester-7/NLP/Assignment-3'):
    """If in Google Colab, switch to relevant working directory

    Returns:
    --------
    bool    True if running in Google Colab, False if not.
    """
    in_google_colab = False
    try:
        from google.colab import drive
        in_google_colab = True
    except:
        pass
    if in_google_colab:
        drive.mount("/content/gdrive")
        os.chdir(gpath)
    return in_google_colab

init_google_colab('/content/gdrive/MyDrive/Assignment-Semester-7/NLP/Assignment-3')

Mounted at /content/gdrive


True

In [4]:
def punc(text):
  exclude = set(string.punctuation)
  text = [''.join(x for x in y if x not in exclude) for y in text]
  text = [x for x in text if x not in stopwords.words('english')]
  return text


In [5]:
from pandas import read_excel
from collections import Counter
data = read_excel("assignment3_data.xlsx")
data.columns = ["Data"]

#Extracting the words

l1 = []
for index, row in data.iterrows():
    words = row["Data"].split(" ")
    for j in words:
        l1.append(j.lower())

#Removing punctuations and stopwords
        
def punc(text):
    exclude = set(string.punctuation)
    text = [''.join(x for x in y if x not in exclude) for y in text]
    text = [x for x in text if x not in stopwords.words('english')]
    return text

l1 = punc(l1)

POS = ['JJ','JJR','JJS','NN','NNS','NNP','NNPS']
temp_vocab = sorted([word for (word,pos) in nltk.pos_tag(l1) if pos in POS])
df1 = pd.DataFrame(columns = ["Words"])
c1 = Counter(temp_vocab)
df1["Words"] = c1.keys()

In [6]:
df1.to_csv("/content/gdrive/MyDrive/Assignment-Semester-7/NLP/Assignment-3/assignment3_data.txt", index = False)

with open("/content/gdrive/MyDrive/Assignment-Semester-7/NLP/Assignment-3/assignment3_data.txt","r+") as f:
    new_f = f.readlines()
    f.seek(0)
    for line in new_f:
        if "What qualities do you think are necessary to be the prime minister of India?" not in line:
            f.write(line)
    f.truncate()

In [7]:
#Give input
with open("assignment3_data.txt") as f:
  lines = f.readlines()

#lines = clean_data(lines)
#model_input=SentGen(lines)
processed = [gensim.utils.simple_preprocess(sentence) for sentence in lines]
#Initialize the model

modelsg = Word2Vec(min_count=1,window=7,size=300,workers=cores-1) 
#skip-gram: sg=1

#Build vocabulary
modelsg.build_vocab(processed)     #Building the vocabulary using entire dataset
vocab_len=len(modelsg.wv.vocab)

#Training the model
modelsg.train(processed, total_examples=modelsg.corpus_count, epochs=50)  #Add callbacks, if required

#Saving the model
modelsg.wv.save_word2vec_format('gensim_w2v_model.bin',binary=True)

#Loading the model during testing time
trained_model= KeyedVectors.load_word2vec_format('gensim_w2v_model.bin', binary=True)

In [8]:
import gensim

modelg = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


In [9]:
modelcb = gensim.models.Word2Vec(processed, min_count=1, size = 100, window = 5, sg =0)#cbow

modelcb.train(processed, total_examples=modelcb.corpus_count, epochs=50)  #Add callbacks, if required

#Saving the model
modelcb.wv.save_word2vec_format('gensim_w2v_model1.bin',binary=True)

#Loading the model during testing time
trained_model1= KeyedVectors.load_word2vec_format('gensim_w2v_model1.bin', binary=True)

In [10]:
from sklearn.cluster import KMeans
model_01 = KMeans(n_clusters=10, 
                  max_iter=10000, random_state=None, n_init=100).fit(
                      X=trained_model.vectors.astype('double'))


model_02 = KMeans(n_clusters=10, 
                  max_iter=10000, random_state=None, n_init=100).fit(
                      X=trained_model1.vectors.astype('double'))

                  
positive_cluster_center = model_01.cluster_centers_[0]
negative_cluster_center = model_01.cluster_centers_[1]

In [11]:
import numpy as np
t1 = []
t2 = []
for i in range(10):
  t1.append(trained_model.similar_by_vector(model_01.cluster_centers_[i], topn=1, restrict_vocab=None))
  t2.append(trained_model1.similar_by_vector(model_02.cluster_centers_[i], topn=1, restrict_vocab=None))

In [12]:
t1

[[('relations', 0.5283156633377075)],
 [('crucial', 0.5405222177505493)],
 [('polite', 0.5564737319946289)],
 [('sense', 0.5592864751815796)],
 [('equality', 0.5225918292999268)],
 [('prejudices', 0.4843869209289551)],
 [('international', 0.46509450674057007)],
 [('hard', 0.42866256833076477)],
 [('motives', 0.4929233193397522)],
 [('umm', 0.505774199962616)]]

In [13]:
t2

[[('political', 0.7111340761184692)],
 [('ulterior', 0.6056719422340393)],
 [('character', 0.597621500492096)],
 [('calm', 0.595472514629364)],
 [('peaceful', 0.5800516605377197)],
 [('decisions', 0.6671245098114014)],
 [('care', 0.6884301900863647)],
 [('hard', 0.6648726463317871)],
 [('determination', 0.6873366236686707)],
 [('humility', 0.6237736940383911)]]

In [14]:
t2 = pd.DataFrame(t2)
t2.to_csv("extracted_words_cbow.txt", index=0)

In [15]:
t1 = pd.DataFrame(t1)
t1.to_csv("extracted_words_sg.txt", index=0)